In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from src.processing.splitting import time_based_split
from src.processing.scaling import scale_features
from src.processing.sequence_creator import create_sequences
from src.training.lstm_training import train_lstm
from src.training.hyperparameters_tuning import tune_hyperparameters

from config import config


df_btc = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "btc.csv",
    parse_dates=["date"],
    index_col="date",
)
df_eth = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "eth.csv",
    parse_dates=["date"],
    index_col="date",
)

## Split


In [2]:
X_train_btc, y_train_btc, X_val_btc, y_val_btc, X_test_btc, y_test_btc = (
    time_based_split(
        df_btc,
        targets=["logPriceChange"],
        test_months=12,
        val_months=3,
        lags=30,
    )
)

X_train_eth, y_train_eth, X_val_eth, y_val_eth, X_test_eth, y_test_eth = (
    time_based_split(
        df_eth,
        targets=["logPriceChange"],
        test_months=12,
        val_months=3,
        lags=30,
    )
)

In [3]:
print(y_test_btc.tail())
print(y_test_btc.head())

            logPriceChange
date                      
2022-12-14       -0.016406
2022-12-15       -0.042190
2022-12-16        0.008828
2022-12-19        0.027989
2022-12-20       -0.005265
            logPriceChange
date                      
2021-12-02       -0.052332
2021-12-03       -0.085608
2021-12-06        0.002319
2021-12-07       -0.003859
2021-12-08       -0.057722


## Scale


In [4]:
X_train_scaled_btc, X_val_scaled_btc, X_test_scaled_btc, scaler_btc = scale_features(
    X_train_btc, X_val_btc, X_test_btc
)

X_train_scaled_eth, X_val_scaled_eth, X_test_scaled_eth, scaler_eth = scale_features(
    X_train_eth, X_val_eth, X_test_eth
)

## Sequence Creation


In [5]:
X_train_seq_btc, y_train_seq_btc = create_sequences(
    X_train_scaled_btc, y_train_btc, seq_length=30
)
X_val_seq_btc, y_val_seq_btc = create_sequences(
    X_val_scaled_btc, y_val_btc, seq_length=30
)
X_test_seq_btc, y_test_seq_btc = create_sequences(
    X_test_scaled_btc, y_test_btc, seq_length=30
)

print(f"X_train_seq_btc shape: {X_train_seq_btc.shape}")
print(f"y_train_seq_btc shape: {y_train_seq_btc.shape}")

X_train_seq_eth, y_train_seq_eth = create_sequences(
    X_train_scaled_eth, y_train_eth, seq_length=30
)
X_val_seq_eth, y_val_seq_eth = create_sequences(
    X_val_scaled_eth, y_val_eth, seq_length=30
)
X_test_seq_eth, y_test_seq_eth = create_sequences(
    X_test_scaled_eth, y_test_eth, seq_length=30
)

X_train_seq_btc shape: (1144, 30, 79)
y_train_seq_btc shape: (1144, 1)


# BTC


### Model Tuning

In [6]:
# study_lstm_btc = tune_hyperparameters(
#     X_train_seq_btc,
#     y_train_seq_btc,
#     X_val_seq_btc,
#     y_val_seq_btc,
#     train_lstm,
#     n_trials=50,
# )

In [7]:
len(y_test_seq_btc)

235

## Model Training

In [8]:
# best parameters from the Optuna study
# best_params_btc = study_lstm_btc.best_params
best_params_btc = {
    "num_lstm_layers": 2,
    "lstm_units_1": 192,
    "lstm_units_2": 64,
    "num_dense_layers": 2,
    "dense_units_1": 32,
    "dense_units_2": 224,
    "dropout_rate": 0.4,
    "learning_rate": 9.492951538251386e-05,
    "batch_size": 32,
}


# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params_btc[f"lstm_units_{i+1}"]
        for i in range(best_params_btc["num_lstm_layers"])
    ],
    "dense_units": [
        best_params_btc[f"dense_units_{i+1}"]
        for i in range(best_params_btc["num_dense_layers"])
    ],
    "dropout_rate": best_params_btc["dropout_rate"],
    "learning_rate": best_params_btc["learning_rate"],
    "batch_size": best_params_btc["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_btc, history_btc = train_lstm(
    X_train_seq_btc,
    y_train_seq_btc,
    X_val_seq_btc,
    y_val_seq_btc,
    params=formatted_params,
)

Epoch 1/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 0.0058 - mae: 0.0581 - val_loss: 0.0110 - val_mae: 0.1013
Epoch 2/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0046 - mae: 0.0516 - val_loss: 0.0012 - val_mae: 0.0282
Epoch 3/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0045 - mae: 0.0501 - val_loss: 8.5539e-04 - val_mae: 0.0239
Epoch 4/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0045 - mae: 0.0504 - val_loss: 0.0012 - val_mae: 0.0278
Epoch 5/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0051 - mae: 0.0502 - val_loss: 0.0040 - val_mae: 0.0564
Epoch 6/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0035 - mae: 0.0448 - val_loss: 9.2938e-04 - val_mae: 0.0242
Epoch 7/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 0.0040 - mae: 0.0469 - val_loss: 0.0020 - val_mae: 0.0365
Epoch 8/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0037 - mae: 0.0476 - val_loss: 0.0022 - val_mae: 0.0393
Epoch 9/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/s

In [9]:
# save weights
model_btc.save_weights(
    config.DATA_DIR
    / "weights"
    / "multi_source_data"
    / "log_price_change_weights_btc.weights.h5"
)

#  generate predictions
y_pred_btc = model_btc.predict(X_test_seq_btc).flatten()

# create new df and save it to csv
predictions_btc = pd.DataFrame(
    {
        "date": y_test_btc.index[30:],
        "predictedLogPriceChange": y_pred_btc,
    }
)


predictions_btc.to_csv(
    config.DATA_DIR
    / "predictions"
    / "multi_source_data"
    / "log_price_change_predictions_btc.csv",
    index=False,
)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


# ETH

### Model Tuning

In [10]:
study_lstm_eth = tune_hyperparameters(
    X_train_seq_eth,
    y_train_seq_eth,
    X_val_seq_eth,
    y_val_seq_eth,
    train_lstm,
    n_trials=50,
)

[I 2025-02-16 15:14:55,157] A new study created in memory with name: no-name-23f0b639-1211-4904-a61f-f432d0c3515c


Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 0.0349 - mae: 0.1223 - val_loss: 0.0042 - val_mae: 0.0538
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0116 - mae: 0.0779 - val_loss: 0.0017 - val_mae: 0.0328
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 0.0097 - mae: 0.0717 - val_loss: 0.0020 - val_mae: 0.0364
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - loss: 0.0063 - mae: 0.0599 - val_loss: 0.0061 - val_mae: 0.0700
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 0.0064 - mae: 0.0593 - val_loss: 0.0146 - val_mae: 0.1146
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 0.0069 - mae: 0.0598 - val_loss: 0.0018 - val_mae: 0.0350
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 0.0046 - mae: 0.0471 - val_loss: 0.0033 - val_mae: 0.0471
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - loss: 0.0049 - mae: 0.0476 - val_loss: 0.0028 - val_mae: 0.0432
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - lo

[I 2025-02-16 15:16:12,351] Trial 0 finished with value: 0.0014744903892278671 and parameters: {'num_lstm_layers': 2, 'lstm_units_1': 256, 'lstm_units_2': 128, 'num_dense_layers': 2, 'dense_units_1': 160, 'dense_units_2': 128, 'dropout_rate': 0.0, 'learning_rate': 0.0012252038599609657, 'batch_size': 32}. Best is trial 0 with value: 0.0014744903892278671.


Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0128 - mae: 0.0816 - val_loss: 0.0018 - val_mae: 0.0321
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0071 - mae: 0.0614 - val_loss: 0.0018 - val_mae: 0.0323
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0054 - mae: 0.0500 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 0.0046 - mae: 0.0451 - val_loss: 0.0015 - val_mae: 0.0311
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.0044 - mae: 0.0440 - val_loss: 0.0015 - val_mae: 0.0311
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0044 - mae: 0.0440 - val_loss: 0.0015 - val_mae: 0.0311
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0043 - mae: 0.0433 - val_loss: 0.0015 - val_mae: 0.0311
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0045 - mae: 0.0442 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - lo

[I 2025-02-16 15:17:32,221] Trial 1 finished with value: 0.0014877213397994637 and parameters: {'num_lstm_layers': 2, 'lstm_units_1': 160, 'lstm_units_2': 192, 'num_dense_layers': 2, 'dense_units_1': 32, 'dense_units_2': 192, 'dropout_rate': 0.4, 'learning_rate': 0.0006251696961024677, 'batch_size': 32}. Best is trial 0 with value: 0.0014744903892278671.


...

Epoch 1/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - loss: 0.0109 - mae: 0.0773 - val_loss: 0.0015 - val_mae: 0.0320
Epoch 2/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 0.0046 - mae: 0.0470 - val_loss: 0.0016 - val_mae: 0.0312
Epoch 3/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - loss: 0.0044 - mae: 0.0444 - val_loss: 0.0015 - val_mae: 0.0311
Epoch 4/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 0.0043 - mae: 0.0442 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 5/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 0.0043 - mae: 0.0438 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 6/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.0043 - mae: 0.0435 - val_loss: 0.0015 - val_mae: 0.0309
Epoch 7/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - loss: 0.0043 - mae: 0.0435 - val_loss: 0.0015 - val_mae: 0.0312
Epoch 8/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - loss: 0.0043 - mae: 0.0437 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 9/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - lo

[I 2025-02-16 17:28:50,516] Trial 48 finished with value: 0.001455384772270918 and parameters: {'num_lstm_layers': 3, 'lstm_units_1': 256, 'lstm_units_2': 160, 'lstm_units_3': 64, 'num_dense_layers': 3, 'dense_units_1': 160, 'dense_units_2': 256, 'dense_units_3': 192, 'dropout_rate': 0.2, 'learning_rate': 0.0016775598442760008, 'batch_size': 16}. Best is trial 45 with value: 0.0011074256617575884.


Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - loss: 0.0073 - mae: 0.0616 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - loss: 0.0064 - mae: 0.0582 - val_loss: 0.0014 - val_mae: 0.0305
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - loss: 0.0051 - mae: 0.0512 - val_loss: 0.0016 - val_mae: 0.0309
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - loss: 0.0053 - mae: 0.0500 - val_loss: 0.0016 - val_mae: 0.0314
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - loss: 0.0049 - mae: 0.0485 - val_loss: 0.0016 - val_mae: 0.0323
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - loss: 0.0048 - mae: 0.0480 - val_loss: 0.0016 - val_mae: 0.0310
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0047 - mae: 0.0469 - val_loss: 0.0015 - val_mae: 0.0309
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - loss: 0.0046 - mae: 0.0463 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - lo

[I 2025-02-16 17:30:01,073] Trial 49 finished with value: 0.0014256677823141217 and parameters: {'num_lstm_layers': 3, 'lstm_units_1': 256, 'lstm_units_2': 192, 'lstm_units_3': 32, 'num_dense_layers': 3, 'dense_units_1': 192, 'dense_units_2': 256, 'dense_units_3': 160, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.00021913992402641905, 'batch_size': 32}. Best is trial 45 with value: 0.0011074256617575884.


Best trial:
  Value (val_loss): 0.0011074256617575884
  Params:
    num_lstm_layers: 3
    lstm_units_1: 224
    lstm_units_2: 160
    lstm_units_3: 96
    num_dense_layers: 3
    dense_units_1: 160
    dense_units_2: 256
    dense_units_3: 128
    dropout_rate: 0.2
    learning_rate: 0.0006776162378486864
    batch_size: 16


## Model Training

In [13]:
# best parameters from the Optuna study
best_params_eth = study_lstm_eth.best_params
 
# best_params_eth = {
#     "num_lstm_layers": 3,
#     "lstm_units_1": 224,
#     "lstm_units_2": 160,
#     "lstm_units_3": 96,
#     "num_dense_layers": 3,
#     "dense_units_1": 160,
#     "dense_units_2": 256,
#     "dropout_rate": 0.2,
#     "learning_rate": 0.0006776162378486864,
#     "batch_size": 16,
# }


# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params_eth[f"lstm_units_{i+1}"] for i in range(best_params_eth["num_lstm_layers"])
    ],
    "dense_units": [
        best_params_eth[f"dense_units_{i+1}"]
        for i in range(best_params_eth["num_dense_layers"])
    ],
    "dropout_rate": best_params_eth["dropout_rate"],
    "learning_rate": best_params_eth["learning_rate"],
    "batch_size": best_params_eth["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_eth, history_eth = train_lstm(
    X_train_seq_eth,
    y_train_seq_eth,
    X_val_seq_eth,
    y_val_seq_eth,
    params=formatted_params,
)

Epoch 1/300
72/72 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - loss: 0.0077 - mae: 0.0646 - val_loss: 0.0015 - val_mae: 0.0311
Epoch 2/300
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - loss: 0.0050 - mae: 0.0511 - val_loss: 0.0016 - val_mae: 0.0320
Epoch 3/300
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.0049 - mae: 0.0478 - val_loss: 0.0018 - val_mae: 0.0321
Epoch 4/300
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 0.0046 - mae: 0.0454 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 5/300
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - loss: 0.0045 - mae: 0.0445 - val_loss: 0.0017 - val_mae: 0.0315
Epoch 6/300
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.0043 - mae: 0.0440 - val_loss: 0.0015 - val_mae: 0.0311
Epoch 7/300
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - loss: 0.0043 - mae: 0.0439 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 8/300
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.0044 - mae: 0.0441 - val_loss: 0.0015 - val_mae: 0.0310
Epoch 9/300
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - lo

In [14]:
# save weights
model_eth.save_weights(
    config.DATA_DIR
    / "weights"
    / "multi_source_data"
    / "log_price_change_weights_eth.weights.h5"
)

#  generate predictions
y_pred_eth = model_eth.predict(X_test_seq_eth).flatten()

# create new df and save it to csv
predictions_eth = pd.DataFrame(
    {
        "date": y_test_eth.index[30:],
        "predictedLogPriceChange": y_pred_eth,
    }
)


predictions_eth.to_csv(
    config.DATA_DIR
    / "predictions"
    / "multi_source_data"
    / "log_price_change_predictions_eth.csv",
    index=False,
)

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
